In [46]:
# Import Dependencies
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy import inspect

In [47]:
# Make a reference to the australian_postcodes_loc.csv file path
csv_path = "Resources/australian_postcodes_loc.csv"

# Import the australian_postcodes_loc.csv file as a DataFrame
postcode_raw_df = pd.read_csv(csv_path, encoding="utf-8")
# Check the csv imported correctly
print(postcode_raw_df.shape)
postcode_raw_df.head()

(18442, 8)


,id,Postcodes,locality,state,lat,long,Lat_precise,Long_precise
0,1,200,ANU,ACT,-35.277700,149.11900,-35.277700,149.119000
1,2,200,Australian National University,ACT,-35.277700,149.11890,-35.277700,149.118527
2,3,800,DARWIN,NT,-12.458684,130.83668,-12.393279,130.776661
3,4,800,DARWIN CITY,NT,-12.458684,130.83668,-12.393279,130.776661
4,5,801,DARWIN,NT,-12.458684,130.83668,-12.463440,130.845642


In [48]:
# Reorganising the columns using double brackets
postcode_raw_df1 = postcode_raw_df[["Postcodes","locality","state","Lat_precise", "Long_precise"]]
postcode_raw_df1.head()

,Postcodes,locality,state,Lat_precise,Long_precise
0,200,ANU,ACT,-35.277700,149.119000
1,200,Australian National University,ACT,-35.277700,149.118527
2,800,DARWIN,NT,-12.393279,130.776661
3,800,DARWIN CITY,NT,-12.393279,130.776661
4,801,DARWIN,NT,-12.463440,130.845642


In [49]:
# Rename columns for readability
postcode_df = postcode_raw_df1.rename(columns={"Postcodes" : "post_code",
                                                "locality": "suburb",
                                                "Lat_precise" : "lat",
                                                "Long_precise" : "long"})
postcode_df.head()

,post_code,suburb,state,lat,long
0,200,ANU,ACT,-35.277700,149.119000
1,200,Australian National University,ACT,-35.277700,149.118527
2,800,DARWIN,NT,-12.393279,130.776661
3,800,DARWIN CITY,NT,-12.393279,130.776661
4,801,DARWIN,NT,-12.463440,130.845642


In [50]:
# Filter down to WA only
postcode_metadata_WAonly_df = postcode_df[(postcode_df["post_code"] >= 6000) &
                            (postcode_df["post_code"] < 7000)]

postcode_metadata_WAonly_df

,post_code,suburb,state,lat,long
15675,6000,CITY DELIVERY CENTRE,WA,-31.955000,115.855000
15676,6000,PERTH,WA,-31.952574,115.861049
15677,6000,PERTH GPO,WA,-31.951300,115.859000
15678,6001,PERTH,WA,-31.950527,115.860457
15679,6003,HIGHGATE,WA,-31.939219,115.869157
...,...,...,...,...,...
17590,6989,MADDINGTON,WA,-32.050286,115.984493
17591,6990,GOSNELLS,WA,-32.082302,115.990267
17592,6991,KELMSCOTT,WA,-32.114400,116.015000
17593,6992,ARMADALE,WA,-32.147301,116.012876


In [51]:
# Add unique index + sort 
postcode_metadata_WAonly_df = postcode_metadata_WAonly_df.sort_values("post_code")
postcode_metadata_WAonly_df = postcode_metadata_WAonly_df.reset_index(drop=True)
postcode_metadata_WAonly_df = postcode_metadata_WAonly_df.reset_index()
postcode_metadata_WAonly_df

,index,post_code,suburb,state,lat,long
0,0,6000,CITY DELIVERY CENTRE,WA,-31.955000,115.855000
1,1,6000,PERTH,WA,-31.952574,115.861049
2,2,6000,PERTH GPO,WA,-31.951300,115.859000
3,3,6001,PERTH,WA,-31.950527,115.860457
4,4,6003,HIGHGATE,WA,-31.939219,115.869157
...,...,...,...,...,...,...
1915,1915,6989,MADDINGTON,WA,-32.050286,115.984493
1916,1916,6990,GOSNELLS,WA,-32.082302,115.990267
1917,1917,6991,KELMSCOTT,WA,-32.114400,116.015000
1918,1918,6992,ARMADALE,WA,-32.147301,116.012876


In [52]:
# Push cleaned DataFrame to a new CSV file
postcode_metadata_WAonly_df.to_csv("Output/postcode_metadata_WAonly_df.csv",
                  encoding="utf-8", index=False, header=True)

In [53]:
# Push to the DB
import config_dm

protocol = 'postgresql'
username = 'postgres'
password = config_dm.postgres_key
host = 'localhost'
port = 5432
database_name = 'project2_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

In [54]:
insp.get_table_names()

['postcode_metadata_waonly', 'solar_metadata_waonly']

In [55]:
postcode_metadata_WAonly_df.to_sql(name='postcode_metadata_waonly', con=engine, if_exists='append', index=False)